In [52]:
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *

In [53]:
chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver'
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'
driver = webdriver.Chrome(chromedriver) 

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/4251084422.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [54]:
df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '종로구_카페 정보.csv'), sep=',')

In [55]:
#df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '종로구_카페 정보.csv'), sep=',')

In [56]:
# df['dong'] = df.iloc[:, 3].str.split(' ').str[1]
# print(df['dong'])

In [57]:
# df.head(10)

In [58]:
df['place_name']

0        토즈모임센터 종로점
1          카페노티드 안국
2           어니언 안국점
3        블루보틀 삼청 카페
4              스태픽스
5             아키비스트
6             안녕인사동
7              산모퉁이
8           포비 광화문점
9            차마시는 뜰
10        테라로사 광화문점
11          프릳츠 원서점
12      카페 레이어드 안국점
13            나무사이로
14             학림다방
15      포비브라이트 광화문점
16             H라운지
17      토즈모임센터 광화문점
18               개뿔
19              청수당
20               부빙
21            카페키이로
22            도트블랭킷
23            카페아르크
24        대충유원지 인왕산
25    카페 하이웨이스트 익선점
26           라카페갤러리
27      서울서 둘째로잘하는집
28            슬로우파크
29             내자상회
Name: place_name, dtype: object

In [59]:
#df.loc[:,'place_name']

In [60]:
#df.replace(df['place_name'], 'ddd')
#df.head(10)

In [61]:
# df['place_name'].replace(' ', 'ddd', inplace=True)
# df['place_name']

In [62]:
#df['place_name'] = df['place_name'].replace(' ', '%20', inplace=True)

In [63]:
# df['naver_keyword'] = df['dong'] + '%20' + df['place_name']
# df['naver_map_url'] = ''
# df.head(10)

In [64]:
df['naver_keyword'] = df['place_name']
df['naver_map_url'] = ''
df.head(10)

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,토즈모임센터 종로점,
1,1,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페,http://place.map.kakao.com/1974266862,카페노티드 안국,
2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,어니언 안국점,
3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,블루보틀 삼청 카페,
4,4,1508329235,스태픽스,서울 종로구 필운동 32-1,서울 종로구 사직로9길 22,0507-1341-2055,카페,음식점 > 카페,http://place.map.kakao.com/1508329235,스태픽스,
5,5,1429224581,아키비스트,서울 종로구 효자동 164-7,서울 종로구 효자로13길 52,02-738-1517,카페,음식점 > 카페,http://place.map.kakao.com/1429224581,아키비스트,
6,6,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,안녕인사동,
7,7,7912367,산모퉁이,서울 종로구 부암동 97-5,서울 종로구 백석동길 153,02-391-4737,카페,음식점 > 카페,http://place.map.kakao.com/7912367,산모퉁이,
8,8,27187492,포비 광화문점,서울 종로구 청진동 246,서울 종로구 종로3길 17,02-2251-8125,카페,음식점 > 카페,http://place.map.kakao.com/27187492,포비 광화문점,
9,9,8082652,차마시는 뜰,서울 종로구 삼청동 35-169,서울 종로구 북촌로11나길 26,02-722-7006,카페,음식점 > 카페 > 전통찻집,http://place.map.kakao.com/8082652,차마시는 뜰,


In [65]:
for i, keyword in enumerate(df['naver_keyword'].tolist()):
#     print(f'{i} {keyword}')
#     print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    print(f'이번에 찾을 키워드 : {i}행 / 전체 {df.shape[0] -1}행중 {keyword}')
    if ' ' in keyword:
        join_keyword = '%20'.join(keyword.split())
    else:
        join_keyword = keyword
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={join_keyword}&sm=hty&style=v5"
        
        print(f'네이버 지도 url 정보: {naver_map_search_url}')

        driver.get(naver_map_search_url)
        time.sleep(2.5)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview').get_attribute('data-cid')
        # df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        time.sleep(1.0)
        
        #만약 검색 결과가 없다면?
    except Exception as e:
        print(e)
        pass
    
driver.quit()

이번에 찾을 키워드 : 0행 / 전체 29행중 토즈모임센터 종로점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=토즈모임센터%20종로점&sm=hty&style=v5
이번에 찾을 키워드 : 1행 / 전체 29행중 카페노티드 안국
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20안국&sm=hty&style=v5
이번에 찾을 키워드 : 2행 / 전체 29행중 어니언 안국점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=어니언%20안국점&sm=hty&style=v5
이번에 찾을 키워드 : 3행 / 전체 29행중 블루보틀 삼청 카페
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=블루보틀%20삼청%20카페&sm=hty&style=v5
이번에 찾을 키워드 : 4행 / 전체 29행중 스태픽스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=스태픽스&sm=hty&style=v5
이번에 찾을 키워드 : 5행 / 전체 29행중 아키비스트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아키비스트&sm=hty&style=v5
이번에 찾을 키워드 : 6행 / 전체 29행중 안녕인사동
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=안녕인사동&sm=hty&style=v5
이번에 찾을 키워드 : 7행 / 전체 29행중 산모퉁이
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=산모퉁이&sm=hty&style=v5
이번에 

In [66]:
df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [67]:
df['naver_map_url']

0       https://m.place.naver.com/restaurant/37532328
1     https://m.place.naver.com/restaurant/1229793109
2     https://m.place.naver.com/restaurant/1240190178
3     https://m.place.naver.com/restaurant/1942921803
4     https://m.place.naver.com/restaurant/1142689405
5     https://m.place.naver.com/restaurant/1783153281
6     https://m.place.naver.com/restaurant/1410492841
7       https://m.place.naver.com/restaurant/12781756
8       https://m.place.naver.com/restaurant/37197875
9       https://m.place.naver.com/restaurant/11719309
10      https://m.place.naver.com/restaurant/38231951
11     https://m.place.naver.com/restaurant/715782489
12     https://m.place.naver.com/restaurant/731605639
13      https://m.place.naver.com/restaurant/33431802
14              https://m.place.naver.com/restaurant/
15    https://m.place.naver.com/restaurant/1730708111
16    https://m.place.naver.com/restaurant/1171532481
17      https://m.place.naver.com/restaurant/37532328
18      https://m.place.nave

In [68]:
df = df.loc[df['naver_map_url'] != 'https://m.place.naver.com/restaurant/']
df

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,토즈모임센터 종로점,https://m.place.naver.com/restaurant/37532328
1,1,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페,http://place.map.kakao.com/1974266862,카페노티드 안국,https://m.place.naver.com/restaurant/1229793109
2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,어니언 안국점,https://m.place.naver.com/restaurant/1240190178
3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,블루보틀 삼청 카페,https://m.place.naver.com/restaurant/1942921803
4,4,1508329235,스태픽스,서울 종로구 필운동 32-1,서울 종로구 사직로9길 22,0507-1341-2055,카페,음식점 > 카페,http://place.map.kakao.com/1508329235,스태픽스,https://m.place.naver.com/restaurant/1142689405
5,5,1429224581,아키비스트,서울 종로구 효자동 164-7,서울 종로구 효자로13길 52,02-738-1517,카페,음식점 > 카페,http://place.map.kakao.com/1429224581,아키비스트,https://m.place.naver.com/restaurant/1783153281
6,6,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,안녕인사동,https://m.place.naver.com/restaurant/1410492841
7,7,7912367,산모퉁이,서울 종로구 부암동 97-5,서울 종로구 백석동길 153,02-391-4737,카페,음식점 > 카페,http://place.map.kakao.com/7912367,산모퉁이,https://m.place.naver.com/restaurant/12781756
8,8,27187492,포비 광화문점,서울 종로구 청진동 246,서울 종로구 종로3길 17,02-2251-8125,카페,음식점 > 카페,http://place.map.kakao.com/27187492,포비 광화문점,https://m.place.naver.com/restaurant/37197875
9,9,8082652,차마시는 뜰,서울 종로구 삼청동 35-169,서울 종로구 북촌로11나길 26,02-722-7006,카페,음식점 > 카페 > 전통찻집,http://place.map.kakao.com/8082652,차마시는 뜰,https://m.place.naver.com/restaurant/11719309


In [69]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

# 네이버 지도의 유형 리스트
naver_map_type_list = []
# # 블로그 리뷰 리스트
# blog_review_list = []
# 블로그 리뷰 수 리스트
blog_review_qty_list = []
# 별점 점수(2022년 종료 예정) 정보 리스트
stars_list = []
# 별점 평가 수(2022년 종료 예정) 리스트
stars_qty_list = []
# 방문자 평가 참여자수 리스트
visitor_review_qty_list = []
# 방문자 리뷰 태그(호평) 리스트
visitor_review_list = ['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
                       '양이 많아요', '혼밥하기 좋아요']


df2 = df[['id', 'place_name', 'address_name', 'phone']]
# df2 = pd.DataFrame(df, columns=['id', 'place_name', 'address_name', 'phone'])



for i in visitor_review_list:
    df2[i] = ''

print(df2)
# print('==============')
# df2.iloc[:, 7]




            id     place_name       address_name           phone 커피가 맛있어요  \
0      8574728     토즈모임센터 종로점   서울 종로구 종로2가 84-8     02-736-0061            
1   1974266862       카페노티드 안국       서울 종로구 재동 86  0507-1337-9973            
2    853028118        어니언 안국점    서울 종로구 계동 146-1   070-7543-2123            
3    353512935     블루보틀 삼청 카페      서울 종로구 소격동 86       1533-6906            
4   1508329235           스태픽스    서울 종로구 필운동 32-1  0507-1341-2055            
5   1429224581          아키비스트   서울 종로구 효자동 164-7     02-738-1517            
6   1779571274          안녕인사동   서울 종로구 관훈동 155-2    02-6954-2910            
7      7912367           산모퉁이    서울 종로구 부암동 97-5     02-391-4737            
8     27187492        포비 광화문점     서울 종로구 청진동 246    02-2251-8125            
9      8082652         차마시는 뜰  서울 종로구 삼청동 35-169     02-722-7006            
10    24725284      테라로사 광화문점      서울 종로구 중학동 19             NaN            
11  1154772851        프릳츠 원서점   서울 종로구 원서동 221-1     02-747-8101            

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/2096811579.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/2096811579.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/2096811579.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [70]:
dict_review = {}
for i in visitor_review_list:
    dict_review[i] = []
    
print(dict_review)

# dict_review ={'커피가 맛있어요' : [], '빵이 맛있어요' : [], '친절해요' : []}

{'커피가 맛있어요': [], '디저트가 맛있어요': [], '빵이 맛있어요': [], '친절해요': [], '인테리어가 멋져요': [], '매장이 청결해요': [], '뷰가 좋아요': [], '음료가 맛있어요': [], '사진이 잘 나와요': [], '특별한 메뉴가 있어요': [], '대화하기 좋아요': [], '가성비가 좋아요': [], '집중하기 좋아요': [], '화장실이 깨끗해요': [], '주차하기 편해요': [], '좌석이 편해요': [], '음식이 맛있어요': [], '매장이 넓어요': [], '단체모임 하기 좋아요': [], '특별한 날 가기 좋아요': [], '재료가 신선해요양이 많아요': [], '혼밥하기 좋아요': []}


In [90]:
chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver' 
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/943689856.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/943689856.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  sub_driver = webdriver.Chrome(chromedriver)


In [91]:
for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    #time.sleep(3.0)
    driver.implicitly_wait(5)
    sub_driver.implicitly_wait(5)
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    #time.sleep(3.0)
    driver.implicitly_wait(10)
    sub_driver.implicitly_wait(10)

    # driver
    
    # 네이버 지도의 유형 분류
    naver_map_type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text
    print(f'naver_map_type: {naver_map_type}')
    naver_map_type_list.append(naver_map_type)
    #time.sleep(2.0)
    driver.implicitly_wait(5)

    # 블로그 리뷰 수
    try:
        blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text
        if not blog_review_qty:
            print('not blog_review_qty ELEMNT!!!')
            blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
    except NoSuchElementException:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
    else:
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
        #time.sleep(2.0)
        driver.implicitly_wait(5)

    # 별점 점수(2022년 종료 예정)
    try:
        star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
        #time.sleep(2.0)
        driver.implicitly_wait(5)
    except:
        stars_list.append('null')


    # sub_driver
    
    # 별점 평가 수(2022년 종료 예정)  //*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]
    try:
        star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]').text
        if not star_review_qty:
            print('not star_review_qty ELEMNT!!!')
            star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]').text
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
    else:
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
        #time.sleep(2.0)
        sub_driver.implicitly_wait(5)


    # 방문자 평가 참여자수
    try:
        visitor_review_qty = sub_driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span").text
        visitor_review_qty = re.sub('[\(\)]', '', visitor_review_qty)
        visitor_review_qty = re.sub('명 참여', '', visitor_review_qty)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        #time.sleep(2.0)
        sub_driver.implicitly_wait(5)
    except:
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)

    # 방문자 호평 부분   # k2tmh   ,   uNsI9
    # visitor_eval_list = sub_driver.find_elements(By.CLASS_NAME, "k2tmh")
    # print(f'방문자 호평 부분: {visitor_eval_list}')

    try:
        visitor_eval_list = sub_driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul")
        print(f'방문자 호평 부분: {visitor_eval_list}')

        for visitor_eval in visitor_eval_list[:5]:
            print('===========================')
            # 평가 전체 내역
            evaluation = visitor_eval.text
            print(f'evaluation: {evaluation}')
            print('===========================')
            print('for문을 통해 평가 내역 추출')
            evaluation = evaluation.split('\n')
            
            exist_review = []
            
            for i in range(0, 14, 3):
                print(evaluation[i])
                text_evaluation = re.sub('["]', '', evaluation[i])
                print('========'*5)
                print(text_evaluation)

                choose_evaluation = evaluation[i+2]
                print(f'choose_evaluation: {choose_evaluation}')
                print('========'*5)
                
                if text_evaluation in visitor_review_list:
                    dict_review[text_evaluation].append(choose_evaluation)
                    exist_review.append(text_evaluation)

#                 if text_evaluation in visitor_review_list:
#                     dict_review[text_evaluation].append(choose_evaluation)
            for review in visitor_review_list:
                if review not in exist_review:
                    dict_review[review].append('')
        #time.sleep(3.0)
        sub_driver.implicitly_wait(5)
    except:
        for i in visitor_review_list:
            dict_review[i].append('')


#     naver_map_type_list.append(naver_map_type)
#     blog_review_qty_list.append(blog_review_qty)
#     stars_list.append(star_review_stars)
#     stars_qty_list.append(star_review_qty)
#     visitor_review_qty_list.append(visitor_review_qty)

#     # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
#     except Exception as e:
#         print(f"{i}행 문제가 발생: {e}")
        
#         # 내역이 없으므로 null을 임시로 넣어줍니다.
#         naver_map_type_list.append('null')
#         blog_review_qty_list.append('null')
#         stars_list.append('null')
#         stars_qty_list.append('null')
#         visitor_review_qty_list.append('null')
        
driver.quit()
sub_driver.quit()

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_51663/3462939726.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):


  0%|          | 0/29 [00:00<?, ?it/s]

naver_map_type: 
not blog_review_qty ELEMNT!!!
블로그 리뷰 수: 
별점 점수(2022년 종료 예정): 


KeyboardInterrupt: 

In [27]:
print(dict_review)
print(len(dict_review['빵이 맛있어요']))

{'커피가 맛있어요': ['0', '1380', '1315', '0', '1609', '0', '777', '0', '515', '699', '0', '0', '0', '2', '0', '0', '86', '0', '45', '150', '0', '0', '0'], '빵이 맛있어요': ['0', '0', '0', '1567', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], '친절해요': ['0', '955', '527', '0', '0', '500', '0', '0', '0', '0', '170', '4', '0', '0', '99', '0', '0', '251', '0', '0'], '인테리어가 멋져요': ['0', '1412', '433', '0', '354', '0', '0', '192', '281', '0', '0', '0', '320', '3', '0', '0', '0', '33', '156', '0', '0', '0'], '매장이 청결해요': ['0', '425', '0', '0', '266', '0', '124', '0', '0', '0', '0', '0', '0', '0', '0', '0'], '뷰가 좋아요': ['0', '420', '0', '0', '0', '167', '0', '0', '0', '0', '0', '0', '47', '0', '0', '0'], '음료가 맛있어요': ['0', '0', '492', '0', '0', '136', '177', '0', '0', '0', '0', '0', '67', '0', '25', '0', '85', '0', '0'], '사진이 잘 나와요': ['0', '653', '0', '0', '0', '0', '0', '0', '5', '0', '0', '0', '20', '74', '0', '0', '0'], '특별한 메뉴가 있어요': ['0', '0', '375', '0', '330', '0', '0', '0', '0', '0', '0', '87', '0'

In [153]:
print(naver_map_type_list)
print(len(naver_map_type_list))

['장소대여', '카페', '카페,디저트', '블루보틀', '카페,디저트', '카페', '복합문화공간', '카페', '베이글', '차', '카페', '카페', '', '카페,디저트', '베이커리', '브런치', '', '갤러리카페', '카페,디저트', '빙수', '카페,디저트', '브런치', '카페,디저트', '카페,디저트', '카페,디저트', '갤러리카페', '차', '이탈리아음식']
28


In [155]:
print(visitor_review_qty_list)
print(len(visitor_review_qty_list))

['9', '3,800', '3,547', '', '583', '2,031', '9', '481', '1,827', '518', '688', '1,044', '1,087', '', '404', '', '9', '14', '699', '319', '213', '585', '9', '103', '437', '9', '745']
27


In [108]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

ValueError: Length of values (1) does not match length of index (3)

In [1]:
print(stars_qty_list)

NameError: name 'stars_qty_list' is not defined

In [17]:
df['naver_map_url'] = ''
df

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,dong,naver_keyword,naver_map_url
0,0,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,종로구,종로구%20토즈모임센터 종로점,
1,1,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페,http://place.map.kakao.com/1974266862,종로구,종로구%20카페노티드 안국,
2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,종로구,종로구%20어니언 안국점,
3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,종로구,종로구%20블루보틀 삼청 카페,
4,4,1508329235,스태픽스,서울 종로구 필운동 32-1,서울 종로구 사직로9길 22,0507-1341-2055,카페,음식점 > 카페,http://place.map.kakao.com/1508329235,종로구,종로구%20스태픽스,
5,5,1429224581,아키비스트,서울 종로구 효자동 164-7,서울 종로구 효자로13길 52,02-738-1517,카페,음식점 > 카페,http://place.map.kakao.com/1429224581,종로구,종로구%20아키비스트,
6,6,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,종로구,종로구%20안녕인사동,
7,7,7912367,산모퉁이,서울 종로구 부암동 97-5,서울 종로구 백석동길 153,02-391-4737,카페,음식점 > 카페,http://place.map.kakao.com/7912367,종로구,종로구%20산모퉁이,
8,8,27187492,포비 광화문점,서울 종로구 청진동 246,서울 종로구 종로3길 17,02-2251-8125,카페,음식점 > 카페,http://place.map.kakao.com/27187492,종로구,종로구%20포비 광화문점,
9,9,8082652,차마시는 뜰,서울 종로구 삼청동 35-169,서울 종로구 북촌로11나길 26,02-722-7006,카페,음식점 > 카페 > 전통찻집,http://place.map.kakao.com/8082652,종로구,종로구%20차마시는 뜰,


In [19]:
df.iloc[0,-1]

''